In [7]:
# 3
import pandas as pd
import datetime
from pmdarima import arima
from pmdarima.arima import auto_arima
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error as mse

def time_series_model_comparison():
    df_initial=pd.read_csv("./data/temperatures.csv",parse_dates=[0])
    dates=pd.date_range(start='1980-01-01', end='2012-12-2', closed='left',freq=pd.offsets.MonthBegin(1))
    df_dates=pd.DataFrame({'ds':dates}) #это объекты типа Timestamp
    df=df_initial.merge(df_dates, on='ds', how='outer').sort_values(by='ds').reset_index(drop=True)
    df=df.fillna(method='ffill')
    #df=df.set_index('ds')

    y_train=df.iloc[:-24]
    y_test=df.iloc[-24:]

    forecasts=pd.DataFrame(index=y_test.index)
    errors=pd.DataFrame(columns=['model','mse'])

    MyArima=auto_arima(y_train['y'].values)
    y_pred=MyArima.predict(24)
    mse_score=mse(y_pred,y_test['y'].values)
    forecasts['arima']=y_pred
    errors=errors.append({'model':'arima','mse':mse_score},ignore_index=True)

    MyProphet=Prophet()
    MyProphet.fit(y_train,algorithm='Newton')
    y_pred=MyProphet.predict(y_test.loc[:,['ds']])['yhat'].values
    mse_score=mse(y_pred,y_test['y'].values)
    forecasts['prophet']=y_pred
    errors=errors.append({'model':'prophet','mse':mse_score},ignore_index=True)

    MyExpSmoothing = ExponentialSmoothing(y_train['y'].values).fit()
    y_pred=MyExpSmoothing.forecast(24)
    mse_score=mse(y_pred,y_test['y'].values)
    forecasts['ex_smoothing']=y_pred
    errors=errors.append({'model':'ex_smoothing','mse':mse_score},ignore_index=True)

    return dict({'trainset':y_train.set_index('ds'), 'testset':y_test.set_index('ds'),\
                 'forecasts':forecasts, 'errors':errors})
otv=time_series_model_comparison()
otv['forecasts']

08:23:59 - cmdstanpy - INFO - Chain [1] start processing
08:24:02 - cmdstanpy - INFO - Chain [1] done processing


,arima,prophet,ex_smoothing
372,-2.240678,-0.774007,-2.814
373,-0.430900,0.158978,-2.814
374,3.376380,3.954034,-2.814
375,8.613397,6.917297,-2.814
376,13.672312,11.377010,-2.814
377,16.915128,15.081854,-2.814
378,17.771337,18.258156,-2.814
379,16.005851,16.939247,-2.814
380,12.064218,14.174062,-2.814
381,7.049794,8.449023,-2.814


In [8]:
otv['forecasts'].iloc[:1]

,arima,prophet,ex_smoothing
372,-2.240678,-0.774007,-2.814
